### Imports

In [16]:
import pandas as pd
import numpy as np
import cv2
import math
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
import os
from PIL import Image
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold

### Semilla

In [17]:
random_seed = 2022

### Hacer resize y guardar una imagen

In [18]:
def resize_and_save_img(src, destination_path):
    original_img = cv2.imread(src)
    old_image_height, old_image_width, channels = original_img.shape
    new_image_width = 60        
    new_image_height = 60
    color = (255,255,255)

    result = np.full((new_image_height, new_image_width, channels), color, dtype=np.uint8)

    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # Centrar imagen
    result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = original_img

    Image.fromarray(result).save(destination_path)

### Unificar data de Train y Test

Se realizará split propio posteriormente (70% TRAIN y 30% TEST)

In [19]:
def generate_new_data():
    train_dir = "Data/Train/"
    test_dir = "Data/Test/"
    destination_dir = "Data_preprocesada/"

    for class_dir in os.listdir(train_dir):
        for train_img in os.listdir(train_dir+class_dir):
            resize_and_save_img(f"{train_dir}{class_dir}/{train_img}", f"{destination_dir}{class_dir}/{train_img}")

    test_info = pd.read_csv("Data/Test.csv")
    for i, test_img in enumerate(sorted(os.listdir(test_dir))):
        resize_and_save_img(f"{test_dir}{test_img}", f"{destination_dir}{test_info.ClassId[i]}/{test_img}")

#generate_new_data() Se corre solo una vez

In [20]:
def get_vector_from_image(image, iterations):
    LL, (LH, HL, HH) = pywt.dwt2(image, 'haar')
    for _ in range(iterations - 1):
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
    return LL.flatten()

#### Obtener data (estratificada y no estratificada)

In [5]:
def get_data(src_dir, iterations):
    x = []
    y = []

    for class_dir in os.listdir(src_dir):
        for train_img in os.listdir(src_dir + class_dir):
            image_path = f"{src_dir}{class_dir}/{train_img}"
            img = Image.open(image_path)
            fv = get_vector_from_image(img, iterations)
            x.append(fv)
            y.append(int(class_dir))
    return np.asarray(x), np.asarray(y)

X, y = get_data(src_dir="Data_preprocesada/", iterations=5)

In [7]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

### Cantidad de muestras por categoria

In [6]:
categories, counts =  np.unique(y, return_counts=True)
dict(zip(categories, counts))

{0: 195,
 1: 2158,
 2: 2466,
 3: 1599,
 4: 2239,
 5: 2210,
 6: 522,
 7: 1640,
 8: 1648,
 9: 1560}

### Cantidad de caracteristicas por muestra

In [9]:
print(len(X[0]))

120


### K-fold Cross Validation Estratificado

In [26]:
def stratified_k_fold_cross_validation(X, y, number_of_folds, random_seed):
    skf = StratifiedKFold(n_splits=number_of_folds, shuffle=True, random_state=random_seed)
    skf.get_n_splits(X, y)

    k_folds = []

    for train_index, test_index in skf.split(X, y):
        fold = {}
        fold['X_train'] = X[train_index]
        fold['X_test'] = X[test_index]
        fold['Y_train'] = y[train_index]
        fold['Y_test'] = y[test_index]
        k_folds.append(fold)
    
    return k_folds

k_folds_data = stratified_k_fold_cross_validation(X, y, 5, random_seed)

### Split de Train y Test

### K-fold Cross Validation no Estratificado

In [3]:
def non_stratified_k_fold_cross_validation(X, y, number_of_folds):
    for train_index, test_index in kf.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]




#### KNN Classifier

In [138]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)